In [1]:
import pymysql
import pandas as pd

In [2]:
# Database connection parameters
host = "localhost"
user = "root"
password = "root"
database = "Stock_Agent_RAG"  

In [3]:
connection = pymysql.connect(host=host, user=user, password=password, database=database)
cursor = connection.cursor()

In [4]:
connection.select_db(database)

In [5]:
# Step 3: Create Table
create_table_query = """
CREATE TABLE IF NOT EXISTS stock_data (
    date DATE,
    name VARCHAR(255),
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    close_adj FLOAT,
    volume BIGINT,
    source VARCHAR(255),
    PRIMARY KEY (date, name)
);
"""

In [6]:
try:
    cursor.execute(create_table_query)
    print("Table 'stock_data' created successfully or already exists.")
except pymysql.Error as e:
    print(f"Error creating table: {e}")

Table 'stock_data' created successfully or already exists.


In [7]:
# Step 4: Load Data from CSV into DataFrame
csv_file_path = "./datasets/processed_stock_dataset.csv"
try:
    df = pd.read_csv(csv_file_path)
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error reading CSV file: {e}")

CSV file loaded successfully.


In [10]:
df.columns

Index(['Unnamed: 0', 'date', 'Name', 'open', 'high', 'low', 'close',
       'adj_close', 'volume', 'source'],
      dtype='object')

In [8]:
insert_query = """
INSERT INTO stock_data (date, name, open, high, low, close, close_adj, volume, source)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [11]:
data_to_insert = [tuple(row) for row in df[['date', 'Name', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'source']].values]

In [12]:
print(data_to_insert[:5])

[('2015-01-02', 'A', 0.01577829887519, 0.015821627143345, 0.0158938300056154, 0.0157213017422051, 0.0145045600680151, 0.0280609189705806, 'Yahoo Finance'), ('2015-01-02', 'AAL', 0.0207988402804839, 0.0209135116783038, 0.0208951099495741, 0.0208977562288033, 0.0198003905717361, 0.1972375056547101, 'Yahoo Finance'), ('2015-01-02', 'AAP', 0.0616415270480183, 0.0622539532843699, 0.0620079920294161, 0.0614757346626321, 0.0552978652891046, 0.0093548629945082, 'Yahoo Finance'), ('2015-01-02', 'AAPL', 0.0106686516705332, 0.0106684393465081, 0.0105647108448502, 0.0105923456158509, 0.0094451756087659, 1.0, 'Yahoo Finance'), ('2015-01-02', 'ABBV', 0.0250758835625526, 0.0254345249332999, 0.0257664361642724, 0.0255429943911946, 0.0168857851619289, 0.0933302641749084, 'Yahoo Finance')]


In [13]:
try:
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()
    print("Data inserted into 'stock_data' table successfully.")
except pymysql.Error as e:
    print(f"Error inserting data: {e}")

Error inserting data: (1062, "Duplicate entry '2015-01-02-AAL' for key 'stock_data.PRIMARY'")


In [14]:
cursor.close()
connection.close()
print("Database connection closed.")

Database connection closed.
